In [1]:
pip install opencv-python mediapipe screen_brightness_control pycaw comtypes

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import cv2
import mediapipe as mp
from math import hypot
from screen_brightness_control import set_brightness
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL

# ✅ Audio Control Setup
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
vol_range = volume.GetVolumeRange()
min_vol, max_vol = vol_range[0], vol_range[1]

# ✅ Mediapipe Hands Setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# ✅ Webcam Setup
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# ✅ Video Recorder Setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame_width, frame_height))

print("✅ Press 'q' in the window to stop recording and close the app.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frameRGB)

    if result.multi_hand_landmarks and result.multi_handedness:
        for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
            label = handedness.classification[0].label  # 'Left' or 'Right'

            # Get thumb and index finger tips
            x1 = int(hand_landmarks.landmark[4].x * w)
            y1 = int(hand_landmarks.landmark[4].y * h)
            x2 = int(hand_landmarks.landmark[8].x * w)
            y2 = int(hand_landmarks.landmark[8].y * h)

            # Draw on hand
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.circle(frame, (x1, y1), 7, (0, 255, 0), cv2.FILLED)
            cv2.circle(frame, (x2, y2), 7, (0, 255, 0), cv2.FILLED)
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Calculate distance
            distance = hypot(x2 - x1, y2 - y1)

            # Volume (Left Hand)
            if label == 'Left':
                vol = np.interp(distance, [30, 300], [min_vol, max_vol])
                volume.SetMasterVolumeLevel(vol, None)
                vol_display = int(np.interp(distance, [30, 300], [0, 100]))
                cv2.putText(frame, f"Volume: {vol_display}%", 
                            (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                print(f"🔊 Volume Set To: {vol_display}%")

            # Brightness (Right Hand)
            else:
                bright = np.interp(distance, [30, 300], [0, 100])
                set_brightness(int(bright))
                cv2.putText(frame, f"Brightness: {int(bright)}%", 
                            (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
                print(f"💡 Brightness Set To: {int(bright)}%")

    # Show and Record Frame
    cv2.imshow("🎥 Hand Gesture Control", frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ✅ Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Video saved as output.avi")


✅ Press 'q' in the window to stop recording and close the app.
💡 Brightness Set To: 61%
💡 Brightness Set To: 66%
💡 Brightness Set To: 68%
💡 Brightness Set To: 68%
💡 Brightness Set To: 65%
💡 Brightness Set To: 65%
💡 Brightness Set To: 64%
💡 Brightness Set To: 66%
💡 Brightness Set To: 67%
💡 Brightness Set To: 66%
💡 Brightness Set To: 65%
💡 Brightness Set To: 64%
💡 Brightness Set To: 64%
💡 Brightness Set To: 64%
💡 Brightness Set To: 62%
💡 Brightness Set To: 61%
💡 Brightness Set To: 63%
💡 Brightness Set To: 62%
💡 Brightness Set To: 61%
💡 Brightness Set To: 61%
💡 Brightness Set To: 60%
💡 Brightness Set To: 60%
💡 Brightness Set To: 60%
💡 Brightness Set To: 59%
💡 Brightness Set To: 60%
💡 Brightness Set To: 60%
💡 Brightness Set To: 60%
💡 Brightness Set To: 60%
💡 Brightness Set To: 59%
💡 Brightness Set To: 55%
💡 Brightness Set To: 55%
💡 Brightness Set To: 49%
💡 Brightness Set To: 45%
💡 Brightness Set To: 46%
💡 Brightness Set To: 46%
💡 Brightness Set To: 48%
💡 Brightness Set To: 54%
💡 Brightness